# Задание 2

Город разбит на квадраты и для каждого квадрата известно количество людей, живущих и работающих на территории этого квадрата. Эти данные находятся в файле `data.csv`. Например, на территории квадрата `85881` живет 101 человек и работает 28 человек.

Также город разбит на административные районы, их существенно меньше. Для каждого квадрата известно то, с какими районами он пересекается и по какой части своей площади. Эти данные находятся в файле `area2district.csv`. Например, примерно 38% площади квадрата `91422` составляет район `55`.

От Вас требуется предложить способ (и реализовать его) расчета количества людей, живущих и работающих в каждом административном районе соответственно. Можно считать, что внутри каждого квадрата люди распределены равномерно.

Требования:
   * В каждом районе должно жить и работать целое неотрицательное количество людей
   * Сумма проживающих и работающих жителей города не должна измениться

В данной задаче нет строго критерия оценивания, но важно сделать упор на скорость работы, в частности не рекомендуется пользоваться циклами `for`. 

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data.csv')
df_dist = pd.read_csv('area2district.csv')

In [3]:
df.sum()

areaid    13901796637
home         19160892
job           9111365
dtype: int64

In [4]:
df_dist.sort_values('districtid').tail()

,areaid,districtid,percent
67903,48148,480,1.0
67656,66057,480,1.0
66692,148136,480,1.0
133588,126063,480,1.0
60593,2445,480,1.0


In [5]:
### Join'им два массива ###

df_joined = df_dist.set_index('areaid').join(df.set_index('areaid'))

In [6]:
df_joined.head()

,districtid,percent,home,job
areaid,,,,
0,120,1.000000,290.0,109.0
1,476,1.000000,16.0,8.0
2,303,0.361579,2.0,0.0
2,90,0.638421,2.0,0.0
3,144,1.000000,NaN,NaN


In [7]:
# Применяем процент для расчета количества людей в каждом районе

df_joined['home_p'] = df_joined['percent']*df_joined['home']
df_joined['job_p'] = df_joined['percent']*df_joined['job']
df_joined.head()

,districtid,percent,home,job,home_p,job_p
areaid,,,,,,
0,120,1.000000,290.0,109.0,290.000000,109.0
1,476,1.000000,16.0,8.0,16.000000,8.0
2,303,0.361579,2.0,0.0,0.723158,0.0
2,90,0.638421,2.0,0.0,1.276842,0.0
3,144,1.000000,NaN,NaN,NaN,NaN


In [8]:
#  Количество людей в районах получилось не целое

df_joined.groupby('districtid')[['home_p', 'job_p']].sum().head()

,home_p,job_p
districtid,,
1,32625.162519,13566.430665
2,3716.605467,535.239616
3,46330.593462,18062.574715
4,3966.771421,1243.848413
5,4833.746494,391.347340


In [9]:
# Ещё раз применим процент, но с округлением, потеряем маленькие "хвостики" на больших данных, которые позже восстановим

df_joined['home_p'] = (df_joined['percent']*df_joined['home']).round()
df_joined['job_p'] = (df_joined['percent']*df_joined['job']).round()
df_joined.head()

,districtid,percent,home,job,home_p,job_p
areaid,,,,,,
0,120,1.000000,290.0,109.0,290.0,109.0
1,476,1.000000,16.0,8.0,16.0,8.0
2,303,0.361579,2.0,0.0,1.0,0.0
2,90,0.638421,2.0,0.0,1.0,0.0
3,144,1.000000,NaN,NaN,NaN,NaN


In [10]:
# Проверка того, что мы потеряли "хвостики округлений"

pd.concat([df[['home', 'job']].sum(), df_joined[['home_p', 'job_p']].sum()])

home      19160892.0
job        9111365.0
home_p    19160875.0
job_p      9111324.0
dtype: float64

In [11]:
# Группировка исходного массива количества людей по квадратом для последующего объединения

df_1 = df.groupby('areaid').sum()
df_1.head()

,home,job
areaid,,
0,290,109
1,16,8
2,2,0
4,18,9
5,34,16


In [12]:
# Группировка расчетного (округленного) массива количества людей в разрезе районов для последующего объединения

df_joined_1 = df_joined.groupby(['areaid'])['districtid', 'home_p', 'job_p'].sum()
df_joined_1.head(6)

,districtid,home_p,job_p
areaid,,,
0,120,290.0,109.0
1,476,16.0,8.0
2,393,2.0,0.0
3,144,0.0,0.0
4,146,18.0,9.0
5,90,34.0,16.0


In [13]:
# Объединяем два ранее подготовленных массива, чтобы посчитать на какое количество целых людей они различаются 
# между собой, те самые потерянные "хвостики"

dfnew = df_joined_1.join(df_1).fillna(0)
dfnew.head()

,districtid,home_p,job_p,home,job
areaid,,,,,
0,120,290.0,109.0,290.0,109.0
1,476,16.0,8.0,16.0,8.0
2,393,2.0,0.0,2.0,0.0
3,144,0.0,0.0,0.0,0.0
4,146,18.0,9.0,18.0,9.0


In [14]:
# Считаем разницу

dfnew['home_p_d'] = dfnew['home'] - dfnew['home_p']
dfnew['job_p_d'] = dfnew['job'] - dfnew['job_p']
dfnew.head(10)

,districtid,home_p,job_p,home,job,home_p_d,job_p_d
areaid,,,,,,,
0,120,290.0,109.0,290.0,109.0,0.0,0.0
1,476,16.0,8.0,16.0,8.0,0.0,0.0
2,393,2.0,0.0,2.0,0.0,0.0,0.0
3,144,0.0,0.0,0.0,0.0,0.0,0.0
4,146,18.0,9.0,18.0,9.0,0.0,0.0
5,90,34.0,16.0,34.0,16.0,0.0,0.0
6,116,0.0,1.0,0.0,1.0,0.0,0.0
7,405,1.0,0.0,1.0,0.0,0.0,0.0
8,39,131.0,28.0,131.0,28.0,0.0,0.0


In [15]:
# Самопроверка, что наш массив рассчитан правильно по сумме различий (потерянные "хвостики")
# home_19160892.0 - home_p_19160875.0 = 17
# job_9111365.0 - job_p_9111324.0     = 41

dfnew[(dfnew['home_p_d'] != 0) | (dfnew['job_p_d'] != 0)][['home_p_d', 'job_p_d']].sum()

home_p_d    17.0
job_p_d     41.0
dtype: float64

In [16]:
# Группировка различий в разрезе квадратов

dfnew_1 = dfnew.groupby('areaid')['home_p_d', 'job_p_d'].sum()
dfnew_1.head()

,home_p_d,job_p_d
areaid,,
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0


In [17]:
# Подготовка исходного объединенного массива в разрезе районов

df_joined_2 = df_joined.reset_index().groupby(['districtid', 'areaid'])['home_p', 'job_p'].sum()
df_joined_2.head()

home_p  job_p
districtid areaid               
1          1158       0.0    0.0
           1289     140.0   64.0
           2558      15.0    8.0
           3190     208.0   83.0
           4374      58.0   55.0

In [18]:
# Объединение массивов исходного в разрезе районов и массива различий в разрезе квадратов, 
# у нас появились повторы значений из массива различий, от которых мы в последствии избавимся
# чтобы получить корректные данные в разрезе районов

df_joined_3 = df_joined_2.join(dfnew_1)
df_joined_3.head()

home_p  job_p  home_p_d  job_p_d
districtid areaid                                  
1          1158       0.0    0.0       0.0      0.0
           1289     140.0   64.0       0.0      0.0
           2558      15.0    8.0       0.0      0.0
           3190     208.0   83.0       0.0      0.0
           4374      58.0   55.0       0.0      0.0

In [19]:
# В качестве меры избавления от повторов значений, описанных выше, воспользуемся изменением порядка 
# и группировкой в разрезе квадратов, но с сохранением районов, далее возьмем только первые значения 
# различий в квадратах по каждому из районов. Таким образом мы отбросим повторы различий, но сохраним 
# привязку к районам.

df_joined_4 = df_joined_3.reorder_levels(['areaid', 'districtid']).sort_values(['areaid', 'home_p']).reset_index().groupby('areaid').first()
df_joined_4 = df_joined_4.reset_index().set_index('districtid')
df_joined_4 = df_joined_4[['home_p_d', 'job_p_d']]
df_joined_4 = df_joined_4.sort_values('districtid')
df_joined_4 = df_joined_4.groupby('districtid').sum()
df_joined_4.head()

,home_p_d,job_p_d
districtid,,
1,1.0,-1.0
2,0.0,0.0
3,0.0,-1.0
4,0.0,-1.0
5,0.0,0.0


In [20]:
# Ещё раз самопроверка, что расчет правильный

df_joined_4.sum()

home_p_d    17.0
job_p_d     41.0
dtype: float64

In [21]:
# Теперь вернемся к группировке исходных данных в разрезе районов, к которым уже будем добавлять различия 
# не по квадратам, а по районам.

df_joined_2 = df_joined.reset_index().groupby(['districtid'])['home_p', 'job_p'].sum()
df_joined_2.head()

,home_p,job_p
districtid,,
1,32627.0,13564.0
2,3712.0,534.0
3,46331.0,18061.0
4,3969.0,1246.0
5,4832.0,392.0


In [22]:
# Теперь объединим исходные данные и данные по разлитчиям, возникшим от округления, в разрезе районов

df_joined_5 = df_joined_2.join(df_joined_4, on='districtid')
df_joined_5.head()

,home_p,job_p,home_p_d,job_p_d
districtid,,,,
1,32627.0,13564.0,1.0,-1.0
2,3712.0,534.0,0.0,0.0
3,46331.0,18061.0,0.0,-1.0
4,3969.0,1246.0,0.0,-1.0
5,4832.0,392.0,0.0,0.0


In [23]:
# Просуммируем объединенные данные и получим скорректированные округленные данные, в которых ранее 
# были потеряны "хвостики" округленгия

df_joined_5['home_p_c'] = df_joined_5['home_p'] + df_joined_5['home_p_d']
df_joined_5['job_p_c'] = df_joined_5['job_p'] + df_joined_5['job_p_d']
df_joined_5.head()

,home_p,job_p,home_p_d,job_p_d,home_p_c,job_p_c
districtid,,,,,,
1,32627.0,13564.0,1.0,-1.0,32628.0,13563.0
2,3712.0,534.0,0.0,0.0,3712.0,534.0
3,46331.0,18061.0,0.0,-1.0,46331.0,18060.0
4,3969.0,1246.0,0.0,-1.0,3969.0,1245.0
5,4832.0,392.0,0.0,0.0,4832.0,392.0


In [24]:
# Самопроверка по сумме, что наши данные по количеству людей в разрезе районов не изменились по отношению к исходным данным

In [25]:
pd.concat([df[['home', 'job']].sum(), df_joined_5[['home_p_c', 'job_p_c']].sum()])

home        19160892.0
job          9111365.0
home_p_c    19160892.0
job_p_c      9111365.0
dtype: float64

In [26]:
# Мы получили корректные данные в массиве df_joined_5[['home_p_c', 'job_p_c']]